In [44]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from scipy.stats import gaussian_kde
from scipy.stats import entropy
import geopandas as gpd
# import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
os.getcwd()

# import pathlib
# temp = pathlib.PosixPath
# pathlib.PosixPath = pathlib.WindowsPath


    
site=78



base_folder = "/project/lhansen/HMC_re/project-amazon/output/sampling/gams/78sites/pa_41.11/"

with open(base_folder+'xi_1/pe_5.3/results.pcl', 'rb') as f:
    # Load the data from the file
    results_adjusted = pickle.load(f)    
beta_hmc =results_adjusted["final_sample_coe"][:,8:]
v_hmc = results_adjusted["V_sample"]

In [38]:
from pysrc.sampling import gibbs_sampling
res = gibbs_sampling()
gamma_mean = res['gamma_mean']
gamma_vcov = res['gamma_vcov']
beta_ori = np.random.multivariate_normal(gamma_mean,gamma_vcov,size=500000)
v_mean = res['V_mean']
v_var   = res['V_var']
v_ori = np.zeros((500000,78))

for i in range(78):
    v_ori[:,i] = np.random.normal(v_mean[i],np.sqrt(v_var[i]),size=500000)

In [45]:
gamma_data = gpd.read_file("/project/lhansen/HMC_re/project-amazon/data/calibration/hmc/"+"gamma_data_site_78.geojson")
X_fit = gamma_data.iloc[:, 0:6].values  # Columns 1 to 6 as X
id_fit = gamma_data.iloc[:, 7].values  # Columns 1 to 6 as X

gamma_baseline= np.exp(X_fit @ beta_ori.T + v_ori[:,].T).T
gamma_test= np.exp(X_fit @ beta_hmc.T + v_hmc[:,].T).T
gamma_hmc=results_adjusted["final_sample"][:, 78:] 

In [ ]:


for i in range(78):
    plt.figure(figsize=(12, 6))
    sns.kdeplot(gamma_baseline[:, i], label=r'baseline', color='red', fill=True, alpha=0.6,linewidth=4) 
    sns.kdeplot(gamma_hmc[:, i], label=r'hmc', color='blue', fill=True, alpha=0.6,linewidth=4)  
    # sns.kdeplot(gamma_test[:, i], label=r'test', color='blue', fill=True, alpha=0.6,linewidth=4)  
    plt.title(r'Density of $\gamma_'+str(i+1)+'$')
    plt.legend()
    # plt.savefig('results/compare/gamma'+str(i+1)+'.png')
    plt.show()


In [46]:
mean_values = np.mean(gamma_hmc, axis=0)
df_mean = pd.DataFrame(mean_values, index=pd.Index(range(1, len(mean_values) + 1), name="Site"))
df_mean.to_csv('mean_post_adj.csv', index=True)

In [18]:
sd_hmc=np.sqrt(np.var(gamma_hmc,ddof=0,axis=0))
quantiles = np.quantile(gamma_hmc, [0.1, 0.5, 0.9], axis=0).T
pd.DataFrame(quantiles).to_csv('quantiles.csv')

In [ ]:
from scipy.stats import gaussian_kde
from scipy.stats import entropy
kl_divergences_gamma = []

for idx in range(site):


    
    kde_func_unadjusted = gaussian_kde(gamma_baseline[:, idx], bw_method='scott')
    kde_func_adjusted = gaussian_kde(gamma_hmc[:, idx], bw_method='scott')

    common_grid = np.linspace(min(gamma_baseline[:, idx].min(), gamma_hmc[:, idx].min()), 
                            max(gamma_baseline[:, idx].max(), gamma_hmc[:, idx].max()), 
                            1000)
    density_unadjusted = kde_func_unadjusted(common_grid)
    density_adjusted = kde_func_adjusted(common_grid)
    density_unadjusted += 1e-20
    density_adjusted += 1e-20
    
    kl_div = entropy(density_unadjusted, density_adjusted)
    print(f'Site {idx+1} KL Divergence: {kl_div}')

    kl_divergences_gamma.append({'Site': idx+1, 'KL_Divergence': kl_div})

kl_df_gamma = pd.DataFrame(kl_divergences_gamma)

# Save the DataFrame to a CSV file
kl_df_gamma.to_csv('kl_gamma.csv', index=False)

In [ ]:
# z_hmc=results_adjusted['sol_val_Z_tracker'][-1][:,:50]
# df_ori = pd.read_csv("hmc_78SitesModel.csv")

# ## Sum the z dot and add it to the z_2017
# site_z_0 = df_ori.iloc[:, 1].to_numpy()
# site_z_0_reshaped = site_z_0.reshape(site, 1)
# site_z_bar=df_ori.iloc[:,2].to_numpy()
# cumulative_hmc_z = (np.cumsum(z_hmc, axis=1)+site_z_0_reshaped/1e9)

# ## load deterministic z solved

# z_det=results_adjusted['sol_val_Z_tracker'][0][:,:50]
# cumulative_det_z = (np.cumsum(z_det, axis=1)+site_z_0_reshaped/1e9)

# ## add t=0 for hmc solution
# cumulative_hmc_z = np.hstack((site_z_0_reshaped/1e9, cumulative_hmc_z))
# cumulative_det_z = np.hstack((site_z_0_reshaped/1e9, cumulative_det_z))



In [ ]:
# percentage_z_hmc=np.sum(cumulative_hmc_z,axis=0)/(np.sum(site_z_bar)/1e9)*100
# percentage_z_det=np.sum(cumulative_det_z,axis=0)/(np.sum(site_z_bar)/1e9)*100
# time = np.arange(1, 51)  


# fig, axes = plt.subplots(1, 1, figsize = (10,8))

# plt.plot(time, percentage_z_det[:50], label="no_uncertainty-78site")
# plt.plot(time, percentage_z_hmc[:50], label=r'$\xi$=1-78site')



# plt.xlabel('Time period')
# plt.ylabel('Z(%)')
# plt.title('Aggregate percentage Z over 50 years')
# plt.grid(False)

# # Display the legend
# plt.legend()
# pdf_pages.savefig(fig)
# plt.show()

In [ ]:
# import geopandas as gpd
# import os
# import matplotlib.pyplot as plt
# from shapely.geometry import Polygon
# clean_amazonBiome = gpd.read_file('map.geojson')
# id=gpd.read_file("id_78.geojson").to_crs(epsg=4326)
# id_hmc = id.copy()
# id_det = id.copy()

In [ ]:
# positions_hmc = []

# for i in range(cumulative_hmc_z.shape[0]):
#     series = cumulative_hmc_z[i, :50]
#     for j in range(1, len(series)):
#         if series[j] < series[j-1] - (0.05 * series[j-1]):

#             positions_hmc.append((j))
#             break  # exit the inner loop once the first position is found for this `i`

# positions_det = []

# for i in range(cumulative_det_z.shape[0]):
#     series = cumulative_det_z[i, :50]
#     for j in range(1, len(series)):
#         if series[j] < series[j-1] - (0.05 * series[j-1]):

#             positions_det.append((j))
#             break  # exit the inner loop once the first position is found for this `i`
# id_hmc['id']=positions_hmc
# id_det['id']=positions_det

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

# # Set up the figure and the gridspec
# fig = plt.figure(figsize=(12, 6))  # you can adjust the figsize to your needs
# gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])  # assuming both plots are of the same width

# # First plot (HMC)
# ax0 = fig.add_subplot(gs[0])

# # Base layer
# id_hmc.boundary.plot(edgecolor='black', facecolor='none', linewidth=0.5, ax=ax0)

# # Amazon Biome overlay
# clean_amazonBiome.boundary.plot(color="darkgreen", linewidth=1.2, ax=ax0)

# for x, y, label in zip(id_hmc.geometry.centroid.x, id_hmc.geometry.centroid.y, id_hmc['id']):
#     ax0.text(x, y, str(label), fontsize=14, ha='center', va='center', fontweight='bold')

# ax0.axis('off')  # turn off the axis
# ax0.set_title('HMC', fontsize=16)  # Title for the first plot

# # Second plot (DET)
# ax1 = fig.add_subplot(gs[1])
# id_det['id'] = positions_det

# # Base layer
# id_det.boundary.plot(edgecolor='black', facecolor='none', linewidth=0.5, ax=ax1)

# # Amazon Biome overlay
# clean_amazonBiome.boundary.plot(color="darkgreen", linewidth=1.2, ax=ax1)

# for x, y, label in zip(id_det.geometry.centroid.x, id_det.geometry.centroid.y, id_det['id']):
#     ax1.text(x, y, str(label), fontsize=14, ha='center', va='center', fontweight='bold')

# ax1.axis('off')  # turn off the axis
# ax1.set_title('DET', fontsize=16)  # Title for the second plot

# plt.tight_layout()
# pdf_pages.savefig(fig)
# plt.show()

In [ ]:
# pdf_pages.close()